Initialization of ES

In [ ]:
from elasticsearch import Elasticsearch

client = Elasticsearch(['host.docker.internal'])
indice = "syslog-2021-02-24,syslog-2021-02-25,syslog-2021-02-26"

1. Query All 

In [ ]:
query = {
    "query": {
        "match_all": {}
    }
}

res = client.search(index=indice, body=query, scroll='100m', size=10000)

print("Got %d Hits:" % res['hits']['total']['value'])
    
sid = res['_scroll_id']
scroll_size = len(res['hits']['hits'])

while scroll_size > 0:
    "Scrolling..."
    print(scroll_size)
    data = client.scroll(scroll_id=sid, scroll='2m')
    # Update the scroll ID
    sid = data['_scroll_id']
    # Get the number of results that returned in the last scroll
    scroll_size = len(data['hits']['hits'])

2. Match Query

In [ ]:
query = {
    "query": {
        "match": {
            "hostname":"for.org"
        }
    }
}

res = client.search(index=indice, body=query)

print("Got %d Hits:" % res['hits']['total']['value'])
for hit in res['hits']['hits']:
    print("%(timestamp)s %(raw)s" % hit["_source"])

3. Multi Match

In [ ]:
query = {
    "query": {
        "multi_match": {
            "query": "up.com ahmadajmi", 
            "fields":["hostname", "application"]
        }
    }
}

res = client.search(index=indice, body=query)

print("Got %d Hits:" % res['hits']['total']['value'])
for hit in res['hits']['hits']:
    print("%(timestamp)s %(raw)s" % hit["_source"])

4. String Query

In [ ]:
query = {
  "query": {
    "query_string": {
      "query": "(for.org) AND (pretty breath) "
    }
  }
}

res = client.search(index=indice, body=query)

print("Got %d Hits:" % res['hits']['total']['value'])
for hit in res['hits']['hits']:
    print("%(timestamp)s %(raw)s" % hit["_source"])

5. Term Query

In [ ]:
query = {
   "query":{
      "term":{"message":"pretty"}
   }
}

res = client.search(index=indice, body=query)

print("Got %d Hits:" % res['hits']['total']['value'])
for hit in res['hits']['hits']:
    print("%(timestamp)s %(raw)s" % hit["_source"])

6. Range Query

In [ ]:
query = {
   "query":{
      "range":{
         "version":{
            "gte":2
         }
      }
   }
}

res = client.search(index=indice, body=query)

print("Got %d Hits:" % res['hits']['total']['value'])
for hit in res['hits']['hits']:
    print("%(timestamp)s %(raw)s" % hit["_source"])

7. Exist Query

In [ ]:
query = {
  "query": {
    "exists": {
      "field": "application"
    }
  }
}

res = client.search(index=indice, body=query)

print("Got %d Hits:" % res['hits']['total']['value'])
for hit in res['hits']['hits']:
    print("%(timestamp)s %(raw)s" % hit["_source"])

8. Regex Query
https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-regexp-query.html

In [ ]:
query = {
  "query": {
    "regexp": {
      "message": {
        "value": "good*",
        "flags": "ALL",
        "max_determinized_states": 10000,
        "rewrite": "constant_score"
      }
    }
  }
}

res = client.search(index=indice, body=query)

print("Got %d Hits:" % res['hits']['total']['value'])
for hit in res['hits']['hits']:
    print("%(timestamp)s %(raw)s" % hit["_source"])

9. Compount Query https://www.elastic.co/guide/en/elasticsearch/reference/current/compound-queries.html

In [ ]:
query = {
   "query": {
      "bool" : {
         "must" : {
            "term" : { "hostname" : "random.net" }
         },
         "should": {
            "term" : { "application" : "ahmadajmi" }
         },
         "minimum_should_match" : 1,
         "boost" : 1.0
      }
   }
}


res = client.search(index=indice, body=query)

print("Got %d Hits:" % res['hits']['total']['value'])
for hit in res['hits']['hits']:
    print("%(timestamp)s %(raw)s" % hit["_source"])